In [1]:

import pandas as pd
import numpy as np

from code_.process_conll import process_file, extract_features
from code_.bert import Tokenizer, convert_to_dataset, compute_metrics, task, batch_size, model_checkpoint
# from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
# from transformers import DataCollatorForTokenClassification
# from datasets import load_metric

c:\Users\snipercapt\Desktop\ANLP\AdvancedNLP\Task_3\code_\bert.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [2]:
df_val = process_file('data/raw/en_ewt-up-dev.conllu')
# df_train = process_file('data/raw/en_ewt-up-train.conllu')
df_test = process_file('data/raw/en_ewt-up-test.conllu')

process_file(): dataframe len: 4979
process_file(): dataframe len: 4802


In [3]:
extract_features(df_val)

,sentences,labels
0,From the AP comes this story :[SEP]comes,3
1,President Bush on Tuesday nominated two indivi...,4
2,President Bush on Tuesday nominated two indivi...,8
3,Bush nominated Jennifer M. Anderson for a 15-y...,1
4,Bush nominated Jennifer M. Anderson for a 15-y...,24
...,...,...
4974,The paint and wheels looked like glass and the...,4
4975,The paint and wheels looked like glass and the...,10
4976,The paint and wheels looked like glass and the...,11
4977,"Also, they have great customer service and a v...",3


In [ ]:
labels_list = [] # to fill all possible gold labels here

In [8]:
dataset = convert_to_dataset(df_train, df_val, df_test)

KeyError: 'label'

In [11]:
import datasets
ds_t = datasets.Dataset.from_pandas(df_train)
ds_ts = datasets.Dataset.from_pandas(df_test)
ds_v = datasets.Dataset.from_pandas(df_val)

ds = datasets.DatasetDict()
ds['train'] = ds_t
ds['test'] = ds_ts

In [12]:
ds['train'].features

{'sentence': Value(dtype='string', id=None),
 'predicate': Value(dtype='string', id=None),
 'predicate index': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
tok = Tokenizer(model_checkpoint, labels_list)

In [ ]:
tokenized_datasets = dataset.map(tok.tokenize_and_align_labels, batched=True)

In [ ]:
# initialise model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list))

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tok.tokenizer)
metric = load_metric("seqeval")

In [ ]:

# labels = [labels_list[i] for i in example[f"{task}_tags"]]
# metric.compute(predictions=[labels], references=[labels])

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tok.tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('model_checkpoints/some_model_name.pth')

In [ ]:
# results, plots, reports etc.

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results